In [1]:
import sys,os
sys.path.append("/home/paco/Documents/site_similarity")

In [2]:
from utils.notebook_utils import load_level_data, create_weighted_nodes, create_graph, draw_graph

In [3]:
lvl_one = load_level_data(level=1)  

Loaded 4238 nodes with records level <= 1 and child size:20335


In [4]:
lvl_one_nodes = create_weighted_nodes(lvl_one)

In [5]:
lvl_one_nodes[:10]

[('villagevoice.com', 'nylon.com', 4.1),
 ('villagevoice.com', 'slantmagazine.com', 2.5),
 ('villagevoice.com', 'gothamist.com', 2.4),
 ('villagevoice.com', 'screendaily.com', 2.1),
 ('villagevoice.com', 'amny.com', 2.1),
 ('insideclimatenews.org', 'wri.org', 12.8),
 ('insideclimatenews.org', 'carbonbrief.org', 12.3),
 ('insideclimatenews.org', 'oneonta.edu', 10.1),
 ('insideclimatenews.org', 'skepticalscience.com', 6.7),
 ('insideclimatenews.org', 'ucsusa.org', 5.2)]

In [6]:
[(k, v, w) for k, v, w in lvl_one_nodes if k == 'politicalmayhem.news']

[('politicalmayhem.news', 'politicalmayhem.news', 0.5)]

In [7]:
from stellargraph import StellarGraph
import pandas as pd


lvl_one_graph = StellarGraph(edges=pd.DataFrame(lvl_one_nodes, columns=['source', 'target', 'weight']))

In [8]:
print(lvl_one_graph.info())

StellarGraph: Undirected multigraph
 Nodes: 11865, Edges: 20399

 Node types:
  default: [11865]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [20399]
        Weights: range=[0.5, 96.2], mean=13.1116, std=12.6289
        Features: none


In [9]:
from stellargraph.data import BiasedRandomWalk

rw = BiasedRandomWalk(lvl_one_graph)

walks = rw.run(
    nodes=list(lvl_one_graph.nodes()),  # root nodes
    length=100,  # maximum length of a random walk
    n=10,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
    weighted=True,  # for weighted random walks
    seed=42,  # random seed fixed for reproducibility
)
print("Number of random walks: {}".format(len(walks)))

Number of random walks: 118650


In [10]:
from gensim.models import Word2Vec

str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, size=64, window=5, min_count=0, sg=1, workers=2, iter=1)

model.save("node2vec_lvl_one_new_data_WEIGHTED_64.model")

In [11]:
model.wv['villagevoice.com'].shape

(64,)

In [12]:
from dataprep.load_annotated_data import apply_splits
from utils.notebook_utils import load_corpus
from modelling.baselines import eval_model

DATA = load_corpus('modified_corpus.csv')
SPLITS = apply_splits(DATA)

print(SPLITS.keys())

train = pd.DataFrame(SPLITS['train-0'])

test = pd.DataFrame(SPLITS['test-0'])

dict_keys(['test-0', 'train-0', 'test-1', 'train-1', 'test-2', 'train-2', 'test-3', 'train-3', 'test-4', 'train-4'])


In [13]:
X_train = train.apply(lambda x: model.wv[x['source_url_processed']], axis=1)

y_train = train.apply(lambda x: x['fact'], axis=1)

In [14]:
X_test = test.apply(lambda x: model.wv[x['source_url_processed']], axis=1)

y_test = test.apply(lambda x: x['fact'], axis=1)

In [15]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score

clf = LogisticRegressionCV(
    Cs=10, cv=10, scoring="accuracy", verbose=True, multi_class="ovr", max_iter=300
)
clf.fit(X_train.values.tolist(), y_train.values.tolist())

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    2.3s finished


LogisticRegressionCV(cv=10, max_iter=300, multi_class='ovr', scoring='accuracy',
                     verbose=True)

In [16]:
y_pred = clf.predict(X_test.values.tolist())

accuracy_score(y_test, y_pred)

0.5761904761904761

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state=42)

In [18]:
clf.fit(X_train.tolist(), y_train.tolist())

GradientBoostingClassifier(random_state=42)

In [19]:
tree_predict = clf.predict(X_test.tolist())
accuracy_score(y_test, tree_predict)

0.5857142857142857

In [20]:
from sklearn import svm

svm_clf = svm.SVC(decision_function_shape='ovo')

svm_clf.fit(X_train.tolist(), y_train.tolist())

SVC(decision_function_shape='ovo')

In [21]:
svm_predict = svm_clf.predict(X_test.tolist())
accuracy_score(y_test, svm_predict)

0.5952380952380952

In [22]:
lin_clf = svm.LinearSVC() # one-vs-rest
lin_clf.fit(X_train.tolist(), y_train.tolist())

LinearSVC()

In [23]:
lin_predict = lin_clf.predict(X_test.tolist())
accuracy_score(y_test, lin_predict)

0.5761904761904761